In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%matplotlib inline
from matplotlib import pyplot as plt

In [3]:
import sys
import os
from subprocess import call 
import pathlib

In [4]:
import numpy as np
import pandas as pd
import xarray as xr

In [5]:
from dask.distributed import Client
from dask.diagnostics import ProgressBar

### parameters for papermill

In [6]:
region = 'IOD'
ndays = 7
roll = 15
climatology = [1991, 2020]
quantiles = [0.1, 0.25, 0.5, 0.75, 0.9]

### create cluster

In [7]:
client = Client(
    n_workers=6, threads_per_worker=4, memory_limit="8GB", local_directory="./dask"
)

In [8]:
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 6
Total threads: 24,Total memory: 44.70 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:33793,Workers: 6
Dashboard: http://127.0.0.1:8787/status,Total threads: 24
Started: Just now,Total memory: 44.70 GiB
Comm: tcp://127.0.0.1:36951,Total threads: 4
Dashboard: http://127.0.0.1:40769/status,Memory: 7.45 GiB
Nanny: tcp://127.0.0.1:45521,


In [9]:
sys.path.append('../code/')

In [10]:
import src

In [11]:
opath = pathlib.Path(f'/media/nicolasf/END19101/data/OISST/daily/{region}')

In [12]:
lfiles = list(opath.glob("sst.day.mean.????.v2.nc"))

In [13]:
lfiles.sort()

In [14]:
lfiles[0]

PosixPath('/media/nicolasf/END19101/data/OISST/daily/IOD/sst.day.mean.1981.v2.nc')

In [15]:
lfiles[-1]

PosixPath('/media/nicolasf/END19101/data/OISST/daily/IOD/sst.day.mean.2022.v2.nc')

In [16]:
dset = xr.open_mfdataset(lfiles, parallel=True, combine_attrs="drop_conflicts", compat="override", coords=['time'])

In [17]:
dset

<xarray.Dataset>
Dimensions:  (time: 14951, lon: 260, lat: 80)
Coordinates:
  * time     (time) datetime64[ns] 1981-09-01 1981-09-02 ... 2022-08-07
  * lon      (lon) float32 45.12 45.38 45.62 45.88 ... 109.1 109.4 109.6 109.9
  * lat      (lat) float32 -9.875 -9.625 -9.375 -9.125 ... 9.375 9.625 9.875
Data variables:
    sst      (time, lat, lon) float32 dask.array<chunksize=(122, 80, 260), meta=np.ndarray>

### need to start a first of January 

In [18]:
# dset = dset.sel(time=slice('1982-01-01',None))

### if regions is 'Ninos' or 'IOD', then first calculates the regional averages

In [19]:
if region == 'Ninos':
    
    dset['sst'] = src.calculates_ninos(dset['sst'], nino='all')
    
elif region == 'IOD': 
    
    dset['sst']= src.calculates_IOD_nodes(dset['sst'], IOD_node='all')

In [20]:
dset

<xarray.Dataset>
Dimensions:  (time: 14951, lon: 260, lat: 80, IOD: 2)
Coordinates:
  * time     (time) datetime64[ns] 1981-09-01 1981-09-02 ... 2022-08-07
  * lon      (lon) float32 45.12 45.38 45.62 45.88 ... 109.1 109.4 109.6 109.9
  * lat      (lat) float32 -9.875 -9.625 -9.375 -9.125 ... 9.375 9.625 9.875
  * IOD      (IOD) object 'IOD_West' 'IOD_East'
Data variables:
    sst      (IOD, time) float32 dask.array<chunksize=(1, 122), meta=np.ndarray>

### remove the 29th of Feb

In [21]:
standard_calendar = dset.time

In [22]:
dset = dset.convert_calendar('noleap')

### calculates the rolling averages if needed 

In [23]:
if ndays is not None: 
    
    dset['sst'] = dset['sst'].rolling({'time':ndays}, center=False, min_periods=ndays).mean('time')
    
    dset = dset.isel(time=slice(ndays+1,None))

In [24]:
dset

<xarray.Dataset>
Dimensions:  (IOD: 2, time: 14933, lon: 260, lat: 80)
Coordinates:
  * time     (time) object 1981-09-09 00:00:00 ... 2022-08-07 00:00:00
  * lon      (lon) float32 45.12 45.38 45.62 45.88 ... 109.1 109.4 109.6 109.9
  * lat      (lat) float32 -9.875 -9.625 -9.375 -9.125 ... 9.375 9.625 9.875
  * IOD      (IOD) object 'IOD_West' 'IOD_East'
Data variables:
    sst      (IOD, time) float64 dask.array<chunksize=(1, 120), meta=np.ndarray>

### now expands the dataset along a dummy dimension 

In [25]:
dset_roll = dset[['sst']].copy()

In [26]:
dset_roll = dset_roll.rolling({'time':roll}, center=True, min_periods=roll).construct(window_dim='roll')

### selects the climatological period 

In [27]:
clim = dset_roll.sel(time=slice(str(climatology[0]), str(climatology[1])))

In [28]:
clim

<xarray.Dataset>
Dimensions:  (time: 10950, IOD: 2, roll: 15)
Coordinates:
  * time     (time) object 1991-01-01 00:00:00 ... 2020-12-31 00:00:00
  * IOD      (IOD) object 'IOD_West' 'IOD_East'
Dimensions without coordinates: roll
Data variables:
    sst      (IOD, time, roll) float64 dask.array<chunksize=(1, 13, 15), meta=np.ndarray>

### transpose, then rechunks 

In [29]:
if region == 'Ninos': 
    
    clim = clim.transpose(*['time','roll','nino'])
    clim = clim.chunk({'time':-1, 'roll':-1, 'nino':len(clim['nino'])})
    
elif region == 'IOD': 

    clim = clim.transpose(*['time','roll','IOD'])
    clim = clim.chunk({'time':-1, 'roll':-1, 'IOD':len(clim['IOD'])})
    
else: 
    
    clim = clim.transpose(*['time','roll','lat','lon'])
    clim = clim.chunk({'time':-1, 'roll':-1, 'lat':2, 'lon':20})

In [30]:
clim

<xarray.Dataset>
Dimensions:  (time: 10950, IOD: 2, roll: 15)
Coordinates:
  * time     (time) object 1991-01-01 00:00:00 ... 2020-12-31 00:00:00
  * IOD      (IOD) object 'IOD_West' 'IOD_East'
Dimensions without coordinates: roll
Data variables:
    sst      (time, roll, IOD) float64 dask.array<chunksize=(10950, 15, 2), meta=np.ndarray>

In [31]:
clim_q = clim.groupby(clim.time.dt.dayofyear).quantile(quantiles, dim=['time','roll'])

In [32]:
clim_ave = clim.groupby(clim.time.dt.dayofyear).mean(dim=['time','roll'])

In [33]:
clim_std = clim.groupby(clim.time.dt.dayofyear).std(dim=['time','roll'])

In [34]:
with ProgressBar(): 
    clim_q = clim_q.compute()
    clim_ave = clim_ave.compute()
    clim_std = clim_std.compute()

In [35]:
clim_q = clim_q.rename({'sst':'quantiles'})

In [36]:
clim_ave

<xarray.Dataset>
Dimensions:    (IOD: 2, dayofyear: 365)
Coordinates:
  * IOD        (IOD) object 'IOD_West' 'IOD_East'
  * dayofyear  (dayofyear) int64 1 2 3 4 5 6 7 8 ... 359 360 361 362 363 364 365
Data variables:
    sst        (IOD, dayofyear) float64 28.03 28.02 28.01 ... 28.59 28.6 28.6

In [37]:
clim_q['average'] = clim_ave['sst']

In [38]:
clim_q

<xarray.Dataset>
Dimensions:    (IOD: 2, dayofyear: 365, quantile: 5)
Coordinates:
  * IOD        (IOD) object 'IOD_West' 'IOD_East'
  * quantile   (quantile) float64 0.1 0.25 0.5 0.75 0.9
  * dayofyear  (dayofyear) int64 1 2 3 4 5 6 7 8 ... 359 360 361 362 363 364 365
Data variables:
    quantiles  (dayofyear, quantile, IOD) float64 27.57 28.19 ... 28.63 28.9
    average    (IOD, dayofyear) float64 28.03 28.02 28.01 ... 28.59 28.6 28.6

In [39]:
clim_q['std'] = clim_std['sst']

In [40]:
opath = pathlib.Path(f'../outputs/{region}')

In [41]:
opath.mkdir(exist_ok=True)

In [42]:
if ndays is not None: 
    
    clim_q.to_zarr(opath.joinpath(f'{region}_OISST_{ndays}days_climatology_{roll}_window.zarr'))

else: 
    
    clim_q.to_zarr(opath.joinpath(f'{region}_OISST_1days_climatology_{roll}_window.zarr'))